# Install Packages

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

# Import Libraries

In [ ]:
import gc
import os
import sys
import shutil
from copy import deepcopy

from PIL import Image
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import efficientnet.tfkeras as efn
import tensorflow as tf
from tensorflow.keras import backend as K 
import tensorflow_hub as tfhub

import torch

from numba import cuda

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sys.path.append('/kaggle/input/weightedboxesfusion')

In [ ]:
from ensemble_boxes import weighted_boxes_fusion, non_maximum_weighted, nms, soft_nms

# Load Data

In [ ]:
def read_prediction_csv(sub_df: pd.DataFrame):
    preds_v = []
    for image_id, preds in zip(sub_df['id'].values, sub_df['PredictionString'].values):
        _cls, bbox, _p_det = [], [], []

        preds_split = preds.split()
        for i in range(0, len(preds_split), 6):
            p_det, x_min, y_min, x_max, y_max = [float(x) for x in preds_split[i + 1:i + 6]]

            if preds_split[i] != 'none':
                bboxes = np.array([x_min, y_min, x_max, y_max])
                _cls.append(1)
                bbox.append(bboxes)
                _p_det.append(p_det)

        preds_v.append(
            {
                'sample_id': image_id,
                'cls': np.array(_cls),
                'bbox': np.array(bbox),
                'p_det': np.array(_p_det),
            }
        )

        del _cls, bbox, p_det

    gc.collect()

    return preds_v

In [ ]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(
        (
            [
                ['00086460a852_study', 'negative 1 0 0 1 1'], 
                ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                ['51759b5579bc_image', 'none 1 0 0 1 1']
            ]
        ), 
        columns=['id', 'PredictionString']
    )
else:
    fast_sub = False

## .dcm to .png

In [ ]:
def read_xray(path, voi_lut: bool = True, fix_monochrome: bool = True):
    dicom = pydicom.read_file(path)

    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array

    if fix_monochrome and dicom.PhotometricInterpretation == 'MONOCHROME1':
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    return data

def resize(array, size, keep_ratio: bool = False, resample=Image.LANCZOS):
    im = Image.fromarray(array)

    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)

    return im

In [ ]:
split = 'test'

save_dir = f'/kaggle/tmp/{split}/'
os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)

## Load study-level image

In [ ]:
STUDY_RES: int = 1024

if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=STUDY_RES)
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))

    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=STUDY_RES)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=STUDY_RES)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

## Load image-level image

In [ ]:
IMAGE_RES: int = 640

image_id = []
dim0 = []
dim1 = []
splits = []

save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)

if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=IMAGE_RES)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)

    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=IMAGE_RES)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=IMAGE_RES)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)

In [ ]:
meta = pd.DataFrame.from_dict(
    {
        'image_id': image_id, 
        'dim0': dim0, 
        'dim1': dim1, 
        'split': splits
    }
)

# Predict study-level image

## TF pipeline

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels: bool = False, target_size=(640, 640), ext: str = 'png'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    return decode


def build_augmenter(img_size: int, with_labels: bool = False):
    def augment(img):
        # img = tf.image.random_crop(value=img, size=(img_size, img_size, 3))
        img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.1)
        return img

    return augment


def build_dataset(
    paths: str, 
    image_size: int,
    bs: int = 16, 
    decode_fn=None,
    augment_fn=None,
    augment: bool = False,
    repeat: bool = False
):
    if decode_fn is None:
        decode_fn = build_decoder(False, (image_size, image_size))

    if augment_fn is None:
        augment_fn = build_augmenter(image_size, False)

    AUTO = tf.data.experimental.AUTOTUNE

    dset = tf.data.Dataset.from_tensor_slices(paths)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    # dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    # dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bs).prefetch(AUTO)

    return dset

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

## Models

In [ ]:
EFNS = [
    efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
    efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7
]

def build_efnet_model(dim: int, ef: int):
    inp = tf.keras.layers.Input(shape=(dim, dim, 3))
    base = EFNS[ef](input_shape=(dim, dim, 3), weights=None, include_top=False)

    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    head = tf.keras.Sequential([tf.keras.layers.Dropout(.5), tf.keras.layers.Dense(4)])

    x1 = head(x)
    x2 = head(x)
    x3 = head(x)
    x4 = head(x)
    x5 = head(x)

    x = (x1 + x2 + x3 + x4 + x5) / 5.
    x = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.Model(inputs=inp, outputs=x)

    return model


def build_efnet_opacity_model(dim: int, ef: int):
    inp = tf.keras.layers.Input(shape=(dim, dim, 3))
    base = EFNS[ef](input_shape=(dim, dim, 3), weights=None, include_top=False)

    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    head = tf.keras.Sequential([tf.keras.layers.Dropout(.5), tf.keras.layers.Dense(1)])

    x1 = head(x)
    x2 = head(x)
    x3 = head(x)
    x4 = head(x)
    x5 = head(x)

    x = (x1 + x2 + x3 + x4 + x5) / 5.
    x = tf.math.sigmoid(x)

    model = tf.keras.Model(inputs=inp, outputs=x)

    return model

## Make format

In [ ]:
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

df['id_last_str'] = [df.loc[i,'id'][-1] for i in range(df.shape[0])]
study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

label_cols = sub_df.columns[2:]

## Inference

In [ ]:
def infer_efnet_recipe(test_paths, model_path: str, ef: int, tta: int, img_size: int, prefix: str, do_fastsub: bool, do_opacity_cls: bool = False):
    global fast_sub

    print(f'[*] recipe ef : {ef} img_size : {img_size} prefix : {prefix}')

    dtest = build_dataset(
        paths=test_paths,
        image_size=img_size,
        bs=BATCH_SIZE, 
        repeat=False if do_fastsub else tta > 1, 
        augment=False if do_fastsub else tta > 1,
        decode_fn=build_decoder(with_labels=False, target_size=(img_size, img_size), ext='png')
    )

    model_paths = sorted(glob(os.path.join(model_path, f'effnet*{ef}-{prefix}-res{img_size}-fold*.h5')))

    model = None
    with strategy.scope():
        if do_opacity_cls:
            model = build_efnet_opacity_model(img_size, ef=ef)
        else:
            model = build_efnet_model(img_size, ef=ef)

    predictions = []
    for model_path in model_paths:
        print(f' [+] load {model_path}')
        with strategy.scope():
            model.load_weights(model_path)

        if do_fastsub:
            pred = model.predict(dtest)
        else:
            pred = model.predict(dtest, steps=tta * len(test_paths) / BATCH_SIZE)[:tta * len(test_paths), :]
            pred = np.mean(pred.reshape(tta, len(test_paths), -1), axis=0)

        predictions.append(pred)

    del model
    del dtest
    del model_paths

    gc.collect()
    K.clear_session()

    return np.mean(predictions, axis=0)

In [ ]:
TTA: int = 1

In [ ]:
pred1 = infer_efnet_recipe(
    test_paths,
    model_path='/kaggle/input/siim-cvoid-19-effnetb7/', 
    ef=7, 
    tta=TTA, 
    img_size=640, 
    prefix='scce0.05-adam-aug_v3',
    do_fastsub=fast_sub
)

In [ ]:
pred2 = infer_efnet_recipe(
    test_paths,
    model_path='/kaggle/input/siim-cvoid-19-effnetb6/', 
    ef=6, 
    tta=TTA, 
    img_size=800, 
    prefix='scce0.05-adam',
    do_fastsub=fast_sub
)

In [ ]:
sub_df[label_cols] = (pred1 + pred2) / 2.

In [ ]:
del pred1, pred2
gc.collect()

In [ ]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on='id', how='left')

## Generate study-string

In [ ]:
for i in range(study_len):
    negative = df.at[i, 'negative']
    typical = df.at[i, 'typical']
    indeterminate = df.at[i, 'indeterminate']
    atypical = df.at[i, 'atypical']

    df.at[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

df_study = df[['id', 'PredictionString']]

## Opacity Detection

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

sub_df = sub_df[study_len:]
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'
sub_df['none'] = 0

label_cols = sub_df.columns[2]

### Inference

In [ ]:
pred1 = infer_efnet_recipe(
    test_paths,
    model_path='/kaggle/input/siim-cvoid-19-effnetb7/', 
    ef=7, 
    tta=1, 
    img_size=640, 
    prefix='scce0.05-adam-aug_v3',
    do_fastsub=fast_sub
)

In [ ]:
pred2 = infer_efnet_recipe(
    test_paths,
    model_path='/kaggle/input/siim-cvoid-19-effnetb6/', 
    ef=6, 
    tta=1, 
    img_size=800, 
    prefix='scce0.05-adam',
    do_fastsub=fast_sub
)

### Ensemble

In [ ]:
preds = (pred1[:, 0] + pred2[:, 0]) / 2.0

sub_df[label_cols] = preds
df_2class = sub_df.reset_index(drop=True)

In [ ]:
del pred1, pred2, preds

K.clear_session()
gc.collect()

In [ ]:
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

# Predict image-level image

In [ ]:
meta = meta[meta['split'] == 'test']

if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on='id', how='left')

In [ ]:
test_dir = f'/kaggle/tmp/{split}/image'

shutil.copytree('/kaggle/input/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

## Utils

### yolo2voc

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    """ 
    bboxes = bboxes.copy().astype(float)  # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]] * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]] * image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]] / 2.
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

### solve_bbox_problems

In [ ]:
def solve_bbox_problems(bbox_v, scores_v):
    bbox_v = np.asarray(bbox_v)
    scores_v = np.asarray(scores_v)

    to_remove = np.zeros(len(bbox_v), dtype=np.bool)
    for i in range(len(bbox_v)):
        x1, y1, x2, y2 = bbox_v[i]

        if x2 < x1:
            x1, x2 = x2, x1
        if y2 < y1:
            y1, y2 = y2, y1
        if x1 < 0:
            x1 = 0
        if x1 > 1:
            x1 = 1
        if x2 < 0:
            x2 = 0
        if x2 > 1:
            x2 = 1
        if y1 < 0:
            y1 = 0
        if y1 > 1:
            y1 = 1
        if y2 < 0:
            y2 = 0
        if y2 > 1:
            y2 = 1
        if (x2 - x1) * (y2 - y1) == 0.0:
            to_remove[i] = True

        bbox_v[i] = x1, y1, x2, y2

    if to_remove.sum() > 0:
        bbox_v[to_remove] = np.array([0.0, 0.0, 1.0, 1.0])
        scores_v[to_remove] = 0.0

    return bbox_v, scores_v

### calc_iou

In [ ]:
def calc_iou(bb0, bb1):
    if len(bb0.shape) == 2:
        bb0 = bb0.T

    if len(bb1.shape) == 2:
        bb1 = bb1.T

    bb0_x0, bb0_y0, bb0_x1, bb0_y1 = bb0
    bb1_x0, bb1_y0, bb1_x1, bb1_y1 = bb1

    # determine the coordinates of the intersection rectangle
    x_left   = np.maximum(bb0_x0, bb1_x0)
    y_top    = np.maximum(bb0_y0, bb1_y0)
    x_right  = np.minimum(bb0_x1, bb1_x1)
    y_bottom = np.minimum(bb0_y1, bb1_y1)

    ret_mask = ~((x_right < x_left) + (y_bottom < y_top))

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb0_area = (bb0_x1 - bb0_x0) * (bb0_y1 - bb0_y0)
    bb1_area = (bb1_x1 - bb1_x0) * (bb1_y1 - bb1_y0)

    iou = intersection_area / (bb0_area + bb1_area - intersection_area)

    return iou * ret_mask

### merge_preds

In [ ]:
def merge_preds(bbox_v, p_det_v=None, mode: str = 'p_det_weight'):
    if p_det_v is None:
        p_det_v = np.ones(bbox_v.shape[0])

    if mode == 'p_det_weight' or mode == 'p_det_weight_pmean':
        typed_p_det_v = p_det_v.astype(bbox_v.dtype)
        p_v = (typed_p_det_v / typed_p_det_v.sum())[:, None]

        bbox = (bbox_v * p_v).sum(axis=0)
        p = p_det_v.mean()
    elif mode == 'p_det_weight_psum':
        typed_p_det_v = p_det_v.astype(bbox_v.dtype)
        p_v = (typed_p_det_v / typed_p_det_v.sum())[:, None]

        bbox = (bbox_v * p_v).sum(axis=0)
        p = p_det_v.sum()
    elif mode == 'median' or mode == 'median_pmean':
        bbox = np.median(bbox_v, axis=0)
        p = p_det_v.mean()
    elif mode == 'p_det_max':
        i_max = p_det_v.argmax()

        bbox = bbox_v[i_max]
        p    = p_det_v[i_max]
    elif mode == 'random':
        i_max = np.random.randint(0, p_det_v.shape[0])

        bbox = bbox_v[i_max]
        p    = p_det_v[i_max]
    else:
        raise ValueError(f'Unknown mode {mode}')

    return bbox, p

### norm_p_det

In [ ]:
def norm_p_det(pred_v):
    p_det_v = [pred_d['p_det'] for pred_d in pred_v if len(pred_d['p_det']) > 0]
    p_det_v = np.concatenate(p_det_v)

    p_det_max = p_det_v.max()

    print(f'[+] p_det_max = {p_det_max}')
    if p_det_max <= 1.0:
        print('[*] skipping norm_p_det')
        return pred_v

    ret_pred_v = deepcopy(pred_v)
    for pred_d in ret_pred_v:
        if len(pred_d['p_det']) > 0:
            pred_d['p_det'] = pred_d['p_det'] / p_det_max

    return ret_pred_v

### fix_boxes

In [ ]:
def fix_boxes(preds_v):
    for preds_d in preds_v:
        if len(preds_d['cls']) > 0:
            dx_dy = preds_d['bbox'][:, 2:] - preds_d['bbox'][:, :2]

            f0 = (dx_dy <= 1).any(axis=-1)
            f1 = (preds_d['p_det'] <= 0) + (preds_d['p_det'] > 1.0)

            if f0.any() or f1.any():
                f = ~(f0 + f1)
                for k in ['p_det', 'bbox', 'cls']:
                    preds_d[k] = preds_d[k][f]

## Detect Yolov5

In [ ]:
import os
import yolov5
from utils.datasets import LoadImages
from utils.general import non_max_suppression, scale_coords, xyxy2xywh

from glob import glob


def yolov5_detect():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model_paths = [
        # YOLOv5x6
        '/kaggle/input/siim-covid19-yolov5/yolov5x6-fold0-mAP0.4466.pt',
        '/kaggle/input/siim-covid19-yolov5/yolov5x6-fold1-mAP0.49373.pt',
        '/kaggle/input/siim-covid19-yolov5/yolov5x6-fold2-mAP0.48003.pt',
        '/kaggle/input/siim-covid19-yolov5/yolov5x6-fold3-mAP0.42454.pt',
        '/kaggle/input/siim-covid19-yolov5/yolov5x6-fold4-mAP0.46058.pt',
        # YOLOv5X6 res640
        # '/kaggle/input/siim-covid19-yolov5x6-res640/yolov5x6-res640-fold0-mAP0.4662.pt',
        # '/kaggle/input/siim-covid19-yolov5x6-res640/yolov5x6-res640-fold1-mAP0.5044.pt',
        # '/kaggle/input/siim-covid19-yolov5x6-res640/yolov5x6-res640-fold2-mAP0.4762.pt',
        # '/kaggle/input/siim-covid19-yolov5x6-res640/yolov5x6-res640-fold3-mAP0.4391.pt',
        # '/kaggle/input/siim-covid19-yolov5x6-res640/yolov5x6-res640-fold4-mAP0.4676.pt',
        # YOLOv5l6
        '/kaggle/input/siim-covid19-yolov5l/yolov5l6-res512-fold0-mAP0.42464.pt',
        '/kaggle/input/siim-covid19-yolov5l/yolov5l6-res512-fold1-mAP0.39763.pt',
        '/kaggle/input/siim-covid19-yolov5l/yolov5l6-res512-fold2-mAP0.42889.pt',
        '/kaggle/input/siim-covid19-yolov5l/yolov5l6-res512-fold3-mAP0.39249.pt',
        '/kaggle/input/siim-covid19-yolov5l/yolov5l6-res512-fold4-mAP0.4241.pt',
        # YOLOv5m6
        '/kaggle/input/siim-covid19-yolov5m/yolov5m6-res512-fold0-mAP0.45113.pt',
        '/kaggle/input/siim-covid19-yolov5m/yolov5m6-res512-fold1-mAP0.44463.pt',
        '/kaggle/input/siim-covid19-yolov5m/yolov5m6-res512-fold2-mAP0.4496.pt',
        '/kaggle/input/siim-covid19-yolov5m/yolov5m6-res512-fold3-mAP0.4121.pt',
        '/kaggle/input/siim-covid19-yolov5m/yolov5m6-res512-fold4-mAP0.42406.pt',
        # YOLOv5s6
        '/kaggle/input/siim-covid19-yolov5s6/yolov5s6-res512-fold0-mAP0.4885.pt',
        '/kaggle/input/siim-covid19-yolov5s6/yolov5s6-res512-fold1-mAP0.4977.pt',
        '/kaggle/input/siim-covid19-yolov5s6/yolov5s6-res512-fold2-mAP0.495.pt',
        '/kaggle/input/siim-covid19-yolov5s6/yolov5s6-res512-fold3-mAP0.456.pt',
        '/kaggle/input/siim-covid19-yolov5s6/yolov5s6-res512-fold4-mAP0.4746.pt',
    ]

    models = [
        torch.load(model_path, map_location=device)['model'].to(device).float().eval()
        for model_path in model_paths
    ]

    dataset = LoadImages('/kaggle/tmp/test/image', img_size=IMAGE_RES)

    all_path = []
    all_bboxes = []
    all_score = []
    for path, img, im0s, _ in dataset:
        img = torch.from_numpy(img).to(device).float() / 255.

        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        bboxes_2, score_2 = [], []
        for model in models:
            pred = model(img, augment=True)[0]
            pred = non_max_suppression(pred, 0.001, 0.5, classes=None, agnostic=False)

            bboxes, score = [], []
            for i, det in enumerate(pred):
                # gain = torch.tensor(im0.shape)[[1, 0, 1, 0]]
                if det is not None and len(det):
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()

                    for *xyxy, conf, _ in det:
                        bboxes.append(torch.tensor(xyxy).view(-1).numpy())
                        score.append(conf.cpu().numpy().item())

            bboxes_2.append(bboxes)
            score_2.append(score)

        all_path.append(path)
        all_score.append(score_2)
        all_bboxes.append(bboxes_2)

    del models
    del dataset
    del model_paths

    gc.collect()
    torch.cuda.empty_cache()

    return all_path, all_score, all_bboxes

## Yolov5

In [ ]:
# def ensemble_pp(boxes, scores, iou_thres: float, skip_box_thr: float):
#     labels = [np.ones(len(scores[idx])) for idx in range(len(scores))]

#     nms_boxes, nms_scores, nms_labels = nms(boxes, scores, labels, weights=None, iou_thr=iou_thres)
#     snms_boxes, snms_scores, snms_labels = soft_nms(boxes, scores, labels, weights=None, iou_thr=iou_thres, sigma=0.1, thresh=skip_box_thr)
#     nmw_boxes, nmw_scores, nmw_labels = non_maximum_weighted(boxes, scores, labels, weights=None, iou_thr=iou_thres, skip_box_thr=skip_box_thr)
#     wbf_boxes, wbf_scores, wbf_labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thres, skip_box_thr=skip_box_thr)

#     del labels

#     boxes, scores, _ = weighted_boxes_fusion(
#         [nms_boxes, snms_boxes, nmw_boxes, wbf_boxes], 
#         [nms_scores, snms_scores, nmw_scores, wbf_scores],
#         [nms_labels, snms_labels, nmw_labels, wbf_labels], 
#         weights=[2, 3, 4, 5], 
#         iou_thr=iou_thres, 
#         skip_box_thr=skip_box_thr
#     )

#     del nms_boxes, nms_scores, nms_labels
#     del snms_boxes, snms_scores, snms_labels
#     del nmw_boxes, nmw_scores, nmw_labels
#     del wbf_boxes, wbf_scores, wbf_labels
#     gc.collect()

#     return boxes, scores

def ensemble_pp(boxes, scores, iou_thres: float, skip_box_thr: float, weights = None):
    boxes, scores, _ = weighted_boxes_fusion(
        boxes, 
        scores,
        [np.ones(len(scores[idx])) for idx in range(len(scores))],
        weights=weights,
        iou_thr=iou_thres, 
        skip_box_thr=skip_box_thr,
    )
    return boxes, scores


# def ensemble_pp(boxes, scores, iou_thres: float, skip_box_thr: float, weights = None):
#     boxes, scores, _ = nms(
#         boxes, 
#         scores,
#         [np.ones(len(scores[idx])) for idx in range(len(scores))],
#         weights=weights,
#         iou_thr=iou_thres, 
#     )
#     return boxes, scores

In [ ]:
with torch.no_grad():
    yolov5_all_path, yolov5_all_score, yolov5_all_bboxes = yolov5_detect()

In [ ]:
yolov5_preds = {}
for row in range(len(yolov5_all_path)):
    image_id = yolov5_all_path[row].split('/')[-1].split('.')[0]
    boxes = yolov5_all_bboxes[row]
    scores = yolov5_all_score[row]

    # normalized to [0, 1]
    boxes = [[coord / (IMAGE_RES - 1) for coord in box] for box in boxes]

    # solve_bbox_problems over the models
    # for i in range(len(boxes)):
    #     boxes, scores = solve_bbox_problems(boxes[i], scores[i])
    boxes, scores = ensemble_pp(
        boxes, 
        scores,
        iou_thres=0.60, 
        skip_box_thr=0.01,
    )

    # unnormalized to [0, IMAGE_RES]
    boxes = [np.asarray([int(coord * (IMAGE_RES - 1)) for coord in box]) for box in boxes]

    yolov5_preds[image_id] = [boxes, scores]

    del image_id, boxes, scores
    gc.collect()

In [ ]:
del yolov5_all_path
del yolov5_all_score
del yolov5_all_bboxes
gc.collect()

## Convert coordinates

In [ ]:
image_ids = []
PredictionStrings = []
for image_id, v in yolov5_preds.items():
    w, h = test_df.loc[test_df['id'] == image_id, ['dim1', 'dim0']].values[0]

    boxes, scores = v

    normalized_boxes = [xyxy2xywh(box[None, :]) / IMAGE_RES for box in boxes]
    rescaled_boxes = [np.round(yolo2voc(h, w, x)[0]) for x in normalized_boxes]
    string_boxes = [
        f'1 {score} {int(box[0])} {int(box[1])} {int(box[2])} {int(box[3])}' 
        for score, box in zip(scores, rescaled_boxes)
    ]

    image_ids.append(image_id)
    PredictionStrings.append(' '.join(string_boxes))

    del boxes, scores
    del normalized_boxes
    del rescaled_boxes
    del string_boxes

pred_df = pd.DataFrame(
    {
        'id': image_ids, 
        'PredictionString': PredictionStrings
    }
)

del image_ids, PredictionStrings
gc.collect()

In [ ]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on='id', how='left').fillna('none 1 0 0 1 1')
sub_df = sub_df[['id', 'PredictionString']]

In [ ]:
for i in range(sub_df.shape[0]):
    prediction_string: str = sub_df.at[i, 'PredictionString']
    if prediction_string == 'none 1 0 0 1 1':
        continue

    sub_df_split = prediction_string.split()

    sub_df_list = []
    for j in range(len(sub_df_split) // 6):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])

    sub_df.at[i, 'PredictionString'] = ' '.join(sub_df_list)

    del sub_df_list
    del prediction_string

## Post-Processing

### Utils

In [ ]:
def clean_predictions(
    preds_v, 
    iou_th: float,
    mode: str = 'p_det_weight',
):
    ret_preds_v = []
    for pred_d in preds_v:
        cls_v = pred_d['cls']
        bbox_v = pred_d['bbox']
        p_det_v = pred_d['p_det']

        new_cls_v = []
        new_bbox_v = []
        new_p_det_v = []
        for i_c in np.unique(cls_v):
            f_c = (cls_v == i_c)

            n_c = f_c.sum()
            if n_c == 1:
                new_cls_v.append(i_c)
                new_bbox_v.append(bbox_v[f_c][0])
                new_p_det_v.append(p_det_v[f_c][0])
            else:
                f_cls_v = cls_v[f_c]
                f_bbox_v = bbox_v[f_c]
                f_p_det_v = p_det_v[f_c]

                to_join_idxs_v = []
                for i in range(n_c):
                    idxs_s = set(np.argwhere(calc_iou(f_bbox_v[i], f_bbox_v) > iou_th).T[0])

                    for i in range(len(to_join_idxs_v)):
                        if len(idxs_s.intersection(to_join_idxs_v[i])) > 0:
                            to_join_idxs_v[i] = to_join_idxs_v[i].union(idxs_s)
                            break
                    else:
                        to_join_idxs_v.append(idxs_s)

                for to_join_idxs in to_join_idxs_v:
                    to_join_idxs = list(to_join_idxs)
                    if len(to_join_idxs) < 1:
                        continue

                    bbox, p_det = merge_preds(f_bbox_v[to_join_idxs], f_p_det_v[to_join_idxs], mode=mode)

                    new_cls_v.append(i_c)
                    new_bbox_v.append(bbox)
                    new_p_det_v.append(p_det)

        ret_preds_d = {
            'cls': np.array(new_cls_v),
            'bbox': np.array(new_bbox_v),
            'p_det': np.array(new_p_det_v),
        }

        for k in pred_d.keys():
            if k not in ['cls', 'bbox', 'p_det']:
                ret_preds_d[k] = pred_d[k]

        ret_preds_v.append(ret_preds_d)

    return ret_preds_v

In [ ]:
def pred_to_str(pred_d):
    bbox_v = pred_d['bbox']
    p_det_v = pred_d['p_det']

    return ' '.join(
        [
            'opacity {:0.05} {} {} {} {}'.format(p_det, *bbox)
            for p_det, bbox in zip(p_det_v, np.round(bbox_v).astype(np.int))
        ]
    )

### Cleaning

In [ ]:
# reference : https://www.kaggle.com/morizin/ensemble331-remake/notebook#Final-cleaning
preds_v = read_prediction_csv(sub_df)

fix_boxes(preds_v)

clean_pred_v = clean_predictions(
    preds_v,
    iou_th=0.60,
    mode='p_det_weight_psum'
)

norm_clean_pred_v = norm_p_det(clean_pred_v)

del preds_v
del clean_pred_v
gc.collect()

In [ ]:
pred_summary_d = {'image_id': [], 'PredictionString': []}
for pred_d in norm_clean_pred_v:
    pred_str: str = pred_to_str(pred_d)

    pred_summary_d['image_id'].append(pred_d['sample_id'])
    pred_summary_d['PredictionString'].append(pred_str)

    del pred_str

sub_df = pd.DataFrame(pred_summary_d)

del pred_summary_d
gc.collect()

### Opacity

In [ ]:
low_threshold: float = 0.00
high_threshold: float = 1.00

sub_df['none'] = df_2class['none']

for i in range(sub_df.shape[0]):
    if sub_df.at[i, 'PredictionString'] != 'none 1 0 0 1 1':
        none_prob: float = sub_df.at[i, 'none']

        sub_df.at[i, 'PredictionString'] = sub_df.at[i, 'PredictionString'] + f' none {none_prob} 0 0 1 1'

        # make sure bbox must be sorted by the confidence
        # todo : threshold
        # todo : remove bbox which has a confidence lower than a none confidence

#         if none_prob < low_threshold:
#             sub_df.at[i, 'PredictionString'] = sub_df.at[i, 'PredictionString']
#             c0 += 1
#         elif low_threshold <= none_prob and none_prob < high_threshold:
#             sub_df.at[i, 'PredictionString'] = sub_df.at[i, 'PredictionString'] + f' none {none_prob} 0 0 1 1'
#             c1 += 1
#         else:
#             sub_df.at[i, 'PredictionString'] = 'none 1 0 0 1 1'
#             c2 += 1

# sub_df = sub_df[['id', 'PredictionString']]
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df = sub_df.rename(columns={'image_id': 'id'})

# Submission

In [ ]:
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
df_study

In [ ]:
shutil.rmtree('/kaggle/working/yolov5')

# EOF